In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import glob

path ="../datasets/csv/" # use your path
all_movies = glob.glob(path + "/*.csv")

li = []

for filename in all_movies:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

all_movies_final = pd.concat(li, axis=0, ignore_index=True)
all_movies_final

,Unnamed: 0,rating,review date,review content
0,0,8.0,8 January 2021,Fern (Frances McDormand) is houseless and livi...
1,1,9.0,24 June 2021,When people put themselves in vulnerable posit...
2,2,6.0,2 August 2021,"""Nomadland"" is a new American movie that premi..."
3,3,7.0,18 February 2021,Nomadland is an exploration of people living i...
4,4,5.0,26 May 2021,"Frances McDormand loses her job in Empire, Nev..."
...,...,...,...,...
49069,504,10.0,5 November 2005,I first saw this movie in the theater when it ...
49070,505,10.0,11 January 2020,I saw this movie on the big screen when it was...
49071,506,10.0,22 June 2019,"Great movie!!!! I love the music, history and ..."
49072,507,10.0,22 November 2005,I am listening to the soundtrack as I pen thes...


In [4]:
data_all = pd.DataFrame(all_movies_final)
print(data_all)

data_all.columns = [c.replace(' ', '_') for c in data_all.columns]

data_all.dropna()

data_all_new = data_all[data_all["rating"] != -1]
data_all_new.head()

       Unnamed: 0  ...                                     review content
0               0  ...  Fern (Frances McDormand) is houseless and livi...
1               1  ...  When people put themselves in vulnerable posit...
2               2  ...  "Nomadland" is a new American movie that premi...
3               3  ...  Nomadland is an exploration of people living i...
4               4  ...  Frances McDormand loses her job in Empire, Nev...
...           ...  ...                                                ...
49069         504  ...  I first saw this movie in the theater when it ...
49070         505  ...  I saw this movie on the big screen when it was...
49071         506  ...  Great movie!!!! I love the music, history and ...
49072         507  ...  I am listening to the soundtrack as I pen thes...
49073         508  ...  The title infers a movie about dancing and a n...

[49074 rows x 4 columns]


,Unnamed:_0,rating,review_date,review_content
0,0,8.0,8 January 2021,Fern (Frances McDormand) is houseless and livi...
1,1,9.0,24 June 2021,When people put themselves in vulnerable posit...
2,2,6.0,2 August 2021,"""Nomadland"" is a new American movie that premi..."
3,3,7.0,18 February 2021,Nomadland is an exploration of people living i...
4,4,5.0,26 May 2021,"Frances McDormand loses her job in Empire, Nev..."


In [5]:
data_all_new['Sentiment'] = data_all_new['rating'].apply(lambda x: "positive" if x>=7 else("negative" if x<=4.9 else "neutral"))
data_all_new

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed:_0,rating,review_date,review_content,Sentiment
0,0,8.0,8 January 2021,Fern (Frances McDormand) is houseless and livi...,positive
1,1,9.0,24 June 2021,When people put themselves in vulnerable posit...,positive
2,2,6.0,2 August 2021,"""Nomadland"" is a new American movie that premi...",neutral
3,3,7.0,18 February 2021,Nomadland is an exploration of people living i...,positive
4,4,5.0,26 May 2021,"Frances McDormand loses her job in Empire, Nev...",neutral
...,...,...,...,...,...
49069,504,10.0,5 November 2005,I first saw this movie in the theater when it ...,positive
49070,505,10.0,11 January 2020,I saw this movie on the big screen when it was...,positive
49071,506,10.0,22 June 2019,"Great movie!!!! I love the music, history and ...",positive
49072,507,10.0,22 November 2005,I am listening to the soundtrack as I pen thes...,positive


In [6]:
data_all_new1 = pd.DataFrame({"review_content1":data_all_new["review_content"], "Sentiment1":data_all_new["Sentiment"]})
data_all_new1.isnull().sum()
data_all_new1.reset_index(drop = True, inplace= True)
data_all_new1

,review_content1,Sentiment1
0,Fern (Frances McDormand) is houseless and livi...,positive
1,When people put themselves in vulnerable posit...,positive
2,"""Nomadland"" is a new American movie that premi...",neutral
3,Nomadland is an exploration of people living i...,positive
4,"Frances McDormand loses her job in Empire, Nev...",neutral
...,...,...
44301,I first saw this movie in the theater when it ...,positive
44302,I saw this movie on the big screen when it was...,positive
44303,"Great movie!!!! I love the music, history and ...",positive
44304,I am listening to the soundtrack as I pen thes...,positive


Data Cleaning 

In [7]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
cleaned_reviews =[]

for i in range(0, 44306):
  review = re.sub('[^a-zA-Z]', ' ', data_all_new1['review_content1'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  cleaned_reviews.append(review)

print(cleaned_reviews)
  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Data Transformation

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1420) 

In [10]:
X = cv.fit_transform(cleaned_reviews).toarray()
# X = np.transpose(np.matrix(x))

y = data_all_new1.iloc[:, -1].values
print(X)
print(y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]]
['positive' 'positive' 'neutral' ... 'positive' 'positive' 'negative']


In [11]:
import pickle

bagofwords_path = 'bow_Sentiment_Model.pkl'
pickle.dump(cv, open(bagofwords_path, "wb"))

Dividing into Training and Testing DataSet

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [13]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [14]:
import joblib
joblib.dump(classifier, 'Classifier_Sentiment_Model')

['/content/drive/Shareddrives/MSBA 22 Study Group/BANA212 Data and Programming Analytics/Group Project/Sentiment Analysis/Classifier_Sentiment_Model']

Model Performance

In [15]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[1049  201   61]
 [ 489  202   96]
 [3560  725 2479]]
              precision    recall  f1-score   support

    negative       0.21      0.80      0.33      1311
     neutral       0.18      0.26      0.21       787
    positive       0.94      0.37      0.53      6764

    accuracy                           0.42      8862
   macro avg       0.44      0.47      0.36      8862
weighted avg       0.76      0.42      0.47      8862

0.4208982171067479
